# 06. PCA (Principal Component Analysis)

## 학습 목표
- PCA의 수학적 원리 이해
- 공분산 행렬과 고유값 분해
- 차원 축소와 정보 손실
- PCA를 numpy로 직접 구현

## 관련 면접 질문
- PCA가 차원을 줄이는 원리는?
- 고유값(eigenvalue)의 의미는?
- PCA를 적용하기 전 데이터 전처리는?

---
## Pre-Quiz

### Q1. PCA의 목적은?
```
당신의 답: 
```

### Q2. 공분산(covariance)이란?
```
당신의 답: 
```

### Q3. 주성분(principal component)이란?
```
당신의 답: 
```

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris, load_digits

np.random.seed(42)

# matplotlib 한글 설정 (macOS)
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False

---
## 1. PCA 직관적 이해

### 핵심 아이디어
- 데이터의 **분산이 최대인 방향**을 찾음
- 그 방향으로 데이터를 투영(projection)
- 분산이 큰 방향 = 정보가 많은 방향

In [ ]:
# 2D 데이터로 PCA 직관 이해
np.random.seed(42)

# 상관관계 있는 2D 데이터 생성
mean = [0, 0]
cov = [[3, 2], [2, 2]]  # 공분산 행렬
X = np.random.multivariate_normal(mean, cov, 200)

# 시각화
plt.figure(figsize=(8, 8))
plt.scatter(X[:, 0], X[:, 1], alpha=0.6)
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('원본 데이터: 어느 방향에 정보가 많을까?')
plt.axis('equal')
plt.grid(True)
plt.show()

---
## 2. PCA 수학적 과정

### Step 1: 데이터 중심화 (Centering)
$$X_{centered} = X - \bar{X}$$

### Step 2: 공분산 행렬 계산
$$C = \frac{1}{n-1} X^T X$$

### Step 3: 고유값 분해 (Eigendecomposition)
$$C v = \lambda v$$
- $v$: 고유벡터 (주성분 방향)
- $\lambda$: 고유값 (해당 방향의 분산)

### Step 4: 투영
$$X_{pca} = X_{centered} \cdot V_k$$
- $V_k$: 상위 k개 고유벡터

In [ ]:
# =================================================
# TODO: PCA 구현
# =================================================

class PCA:
    def __init__(self, n_components):
        """
        Args:
            n_components: 축소할 차원 수
        """
        self.n_components = n_components
        self.components_ = None      # 주성분 (고유벡터)
        self.explained_variance_ = None  # 고유값
        self.mean_ = None            # 데이터 평균
    
    def fit(self, X):
        """
        PCA 학습: 고유벡터, 고유값 계산
        
        Args:
            X: (n_samples, n_features)
        """
        # ============================================
        # 여기에 구현하세요
        # 1. 평균 계산 및 저장
        # 2. 데이터 중심화
        # 3. 공분산 행렬 계산
        # 4. 고유값 분해: np.linalg.eigh() 사용
        # 5. 고유값 내림차순 정렬
        # 6. 상위 n_components개 저장
        # ============================================
        pass
    
    def transform(self, X):
        """
        데이터를 주성분 공간으로 투영
        
        Args:
            X: (n_samples, n_features)
        Returns:
            X_pca: (n_samples, n_components)
        """
        # ============================================
        # 여기에 구현하세요
        # 1. 중심화: X - mean
        # 2. 투영: X_centered @ components.T
        # ============================================
        pass
    
    def fit_transform(self, X):
        """fit과 transform을 한 번에"""
        self.fit(X)
        return self.transform(X)
    
    def explained_variance_ratio(self):
        """각 주성분이 설명하는 분산 비율"""
        return self.explained_variance_ / np.sum(self.explained_variance_)

In [ ]:
# ============ 테스트 ============
def test_pca():
    np.random.seed(42)
    
    # 간단한 테스트 데이터
    X = np.random.randn(100, 5)
    
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X)
    
    assert X_pca.shape == (100, 2), f"Shape 오류: {X_pca.shape}"
    assert pca.components_.shape == (2, 5), f"Components shape 오류: {pca.components_.shape}"
    
    # sklearn과 비교
    from sklearn.decomposition import PCA as SklearnPCA
    sklearn_pca = SklearnPCA(n_components=2)
    X_sklearn = sklearn_pca.fit_transform(X)
    
    # 부호는 다를 수 있으므로 절대값 비교
    assert np.allclose(np.abs(X_pca), np.abs(X_sklearn), atol=1e-10), "sklearn과 결과 불일치"
    
    print("✅ PCA 테스트 통과!")
    print(f"   설명된 분산 비율: {pca.explained_variance_ratio()}")

test_pca()

---
## 3. 주성분 시각화

In [ ]:
# 2D 데이터에서 주성분 방향 시각화
def visualize_principal_components():
    np.random.seed(42)
    
    mean = [0, 0]
    cov = [[3, 2], [2, 2]]
    X = np.random.multivariate_normal(mean, cov, 200)
    
    pca = PCA(n_components=2)
    pca.fit(X)
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # 원본 데이터 + 주성분 방향
    axes[0].scatter(X[:, 0], X[:, 1], alpha=0.5)
    
    # 주성분 화살표 그리기
    origin = pca.mean_
    for i, (component, var) in enumerate(zip(pca.components_, pca.explained_variance_)):
        scale = np.sqrt(var) * 2  # 분산에 비례한 길이
        axes[0].annotate('', xy=origin + component * scale, xytext=origin,
                        arrowprops=dict(arrowstyle='->', color='red', lw=3))
        axes[0].text(origin[0] + component[0] * scale * 1.1,
                    origin[1] + component[1] * scale * 1.1,
                    f'PC{i+1} (var={var:.2f})', fontsize=12)
    
    axes[0].set_xlabel('Feature 1')
    axes[0].set_ylabel('Feature 2')
    axes[0].set_title('원본 데이터 + 주성분 방향')
    axes[0].axis('equal')
    axes[0].grid(True)
    
    # 변환된 데이터
    X_pca = pca.transform(X)
    axes[1].scatter(X_pca[:, 0], X_pca[:, 1], alpha=0.5, c='green')
    axes[1].axhline(y=0, color='r', linestyle='--')
    axes[1].axvline(x=0, color='r', linestyle='--')
    axes[1].set_xlabel('PC1 (분산 최대)')
    axes[1].set_ylabel('PC2 (PC1과 직교)')
    axes[1].set_title('PCA 변환 후')
    axes[1].axis('equal')
    axes[1].grid(True)
    
    plt.tight_layout()
    plt.show()

visualize_principal_components()

---
## 4. Iris 데이터셋에 PCA 적용

In [ ]:
# Iris 데이터셋 (4D → 2D)
iris = load_iris()
X_iris = iris.data
y_iris = iris.target

print(f"원본 데이터: {X_iris.shape}")
print(f"Features: {iris.feature_names}")

# PCA 적용
pca_iris = PCA(n_components=2)
X_iris_pca = pca_iris.fit_transform(X_iris)

print(f"\nPCA 후: {X_iris_pca.shape}")
print(f"설명된 분산: {pca_iris.explained_variance_ratio()}")
print(f"총 설명된 분산: {sum(pca_iris.explained_variance_ratio()):.2%}")

In [ ]:
# Iris PCA 시각화
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# PCA 결과
colors = ['red', 'green', 'blue']
for i, (color, target_name) in enumerate(zip(colors, iris.target_names)):
    mask = y_iris == i
    axes[0].scatter(X_iris_pca[mask, 0], X_iris_pca[mask, 1], 
                   c=color, label=target_name, alpha=0.7)

axes[0].set_xlabel(f'PC1 ({pca_iris.explained_variance_ratio()[0]:.1%})')
axes[0].set_ylabel(f'PC2 ({pca_iris.explained_variance_ratio()[1]:.1%})')
axes[0].set_title('Iris: PCA (4D → 2D)')
axes[0].legend()
axes[0].grid(True)

# 설명된 분산 누적
pca_full = PCA(n_components=4)
pca_full.fit(X_iris)
cumsum = np.cumsum(pca_full.explained_variance_ratio())

axes[1].bar(range(1, 5), pca_full.explained_variance_ratio(), alpha=0.7, label='개별')
axes[1].plot(range(1, 5), cumsum, 'ro-', label='누적')
axes[1].axhline(y=0.95, color='g', linestyle='--', label='95% 기준')
axes[1].set_xlabel('주성분 번호')
axes[1].set_ylabel('설명된 분산 비율')
axes[1].set_title('Scree Plot')
axes[1].legend()
axes[1].set_xticks(range(1, 5))

plt.tight_layout()
plt.show()

---
## 5. MNIST 데이터셋에 PCA 적용

In [ ]:
# MNIST digits (64D → 2D)
digits = load_digits()
X_digits = digits.data
y_digits = digits.target

print(f"원본 데이터: {X_digits.shape} (8x8 이미지 = 64 pixels)")

# PCA 적용
pca_digits = PCA(n_components=2)
X_digits_pca = pca_digits.fit_transform(X_digits)

print(f"PCA 후: {X_digits_pca.shape}")
print(f"설명된 분산: {sum(pca_digits.explained_variance_ratio()):.2%}")

In [ ]:
# MNIST PCA 시각화
plt.figure(figsize=(12, 10))

scatter = plt.scatter(X_digits_pca[:, 0], X_digits_pca[:, 1], 
                     c=y_digits, cmap='tab10', alpha=0.6, s=20)
plt.colorbar(scatter, ticks=range(10))
plt.xlabel(f'PC1 ({pca_digits.explained_variance_ratio()[0]:.1%})')
plt.ylabel(f'PC2 ({pca_digits.explained_variance_ratio()[1]:.1%})')
plt.title('MNIST Digits: PCA (64D → 2D)')
plt.grid(True, alpha=0.3)
plt.show()

---
## 6. 주성분 개수 선택하기

In [ ]:
# =================================================
# TODO: 주어진 분산 비율을 달성하는 최소 주성분 개수 찾기
# =================================================

def find_n_components_for_variance(X, target_variance=0.95):
    """
    목표 분산 비율을 달성하는 최소 주성분 개수 찾기
    
    Args:
        X: 데이터
        target_variance: 목표 설명 분산 비율 (0-1)
    Returns:
        n_components: 필요한 주성분 개수
    """
    # ============================================
    # 여기에 구현하세요
    # 1. 전체 주성분으로 PCA fit
    # 2. 누적 분산 비율 계산
    # 3. target_variance 이상인 최소 개수 찾기
    # ============================================
    pass

In [ ]:
# 테스트
n_95 = find_n_components_for_variance(X_digits, 0.95)
n_99 = find_n_components_for_variance(X_digits, 0.99)

print(f"MNIST (64D)에서:")
print(f"  95% 분산 유지: {n_95}개 주성분 필요")
print(f"  99% 분산 유지: {n_99}개 주성분 필요")

---
## Post-Quiz

### Q1. PCA 전에 데이터 표준화(StandardScaler)가 필요한 이유는?
```
당신의 답: 
```

### Q2. 고유값이 큰 주성분을 먼저 선택하는 이유는?
```
당신의 답: 
```

### Q3. PCA의 한계점은?
```
당신의 답: 
```

---
## 정답

<details>
<summary>👉 클릭하여 정답 코드 보기</summary>

```python
class PCA:
    def __init__(self, n_components):
        self.n_components = n_components
        self.components_ = None
        self.explained_variance_ = None
        self.mean_ = None
    
    def fit(self, X):
        # 1. 평균 계산
        self.mean_ = np.mean(X, axis=0)
        
        # 2. 중심화
        X_centered = X - self.mean_
        
        # 3. 공분산 행렬
        cov_matrix = np.cov(X_centered.T)
        
        # 4. 고유값 분해
        eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)
        
        # 5. 내림차순 정렬
        idx = np.argsort(eigenvalues)[::-1]
        eigenvalues = eigenvalues[idx]
        eigenvectors = eigenvectors[:, idx]
        
        # 6. 상위 n_components개 선택
        self.components_ = eigenvectors[:, :self.n_components].T
        self.explained_variance_ = eigenvalues[:self.n_components]
        
        return self
    
    def transform(self, X):
        X_centered = X - self.mean_
        return X_centered @ self.components_.T
    
    def fit_transform(self, X):
        self.fit(X)
        return self.transform(X)
    
    def explained_variance_ratio(self):
        return self.explained_variance_ / np.sum(self.explained_variance_)


def find_n_components_for_variance(X, target_variance=0.95):
    n_features = X.shape[1]
    pca = PCA(n_components=n_features)
    pca.fit(X)
    
    cumsum = np.cumsum(pca.explained_variance_ratio())
    n_components = np.argmax(cumsum >= target_variance) + 1
    
    return n_components
```

</details>

<details>
<summary>👉 Quiz 정답</summary>

### Pre-Quiz

**Q1. PCA의 목적**
고차원 데이터의 차원을 줄이면서 정보를 최대한 보존. 분산이 최대인 방향(주성분)을 찾아 데이터를 투영.

**Q2. 공분산**
두 변수가 함께 변하는 정도. cov(X,Y) > 0이면 양의 상관, < 0이면 음의 상관. 공분산 행렬은 모든 변수 쌍의 공분산을 담음.

**Q3. 주성분**
데이터의 분산이 최대인 방향. 공분산 행렬의 고유벡터가 주성분 방향, 고유값이 해당 방향의 분산.

### Post-Quiz

**Q1. 표준화가 필요한 이유**
PCA는 분산 기반. 스케일이 큰 feature가 주성분을 지배할 수 있음. 표준화로 모든 feature를 동등하게 취급.

**Q2. 고유값 큰 순서대로 선택 이유**
고유값 = 해당 주성분 방향의 분산. 분산이 큰 방향 = 데이터 변동성이 큰 방향 = 정보가 많은 방향. 정보 손실 최소화.

**Q3. PCA 한계점**
1) **선형 변환만 가능** - 비선형 구조 못 잡음 (→ t-SNE, UMAP)
2) **분산 기반** - 분산이 중요하지 않은 경우 부적합
3) **해석의 어려움** - 주성분이 원래 feature의 조합이라 의미 해석 어려움
4) **이상치에 민감** - 공분산이 이상치에 영향 받음

</details>